In [31]:
!pip install -q bitsandbytes accelerate
!pip install -q --upgrade transformers peft datasets
!pip install -q huggingface_hub


In [32]:
import torch
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    Trainer,
    TrainingArguments,
    DataCollatorForLanguageModeling
)
from peft import (
    get_peft_model,
    LoraConfig,
    TaskType,
    prepare_model_for_kbit_training
)
from datasets import load_dataset, Dataset
from huggingface_hub import notebook_login

In [43]:
MODEL_NAME = "Qwen/Qwen2.5-1.5B-Instruct"
OUTPUT_DIR = "./qwen_world_history"
HF_REPO_NAME = "loaimo/qwen-world-history-lora"  # <-- Change this!
MAX_LENGTH = 384

In [ ]:
login(token="")

In [35]:
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print("Loading model with 8-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    quantization_config=bnb_config,
    torch_dtype=torch.bfloat16,
    pad_token_id=tokenizer.pad_token_id,
)

Loading tokenizer...
Loading model with 8-bit quantization...


In [36]:
test_prompt = """Question: What was the impact of Winston Churchill's 1946 speech on U.S. public opinion towards the Soviet Union?

Answer:"""

def generate_response(prompt, model, tokenizer, max_new_tokens=200):
    """Generate a response from the model"""
    model.eval()
    with torch.no_grad():
        tokens = tokenizer(prompt, return_tensors="pt")['input_ids'].to("cuda")
        output = model.generate(
            tokens,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
            eos_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1,
        )
    return tokenizer.decode(output[0], skip_special_tokens=True)

print("=" * 50)
print("BEFORE TRAINING:")
print("=" * 50)
print(generate_response(test_prompt, model, tokenizer))

BEFORE TRAINING:


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Question: What was the impact of Winston Churchill's 1946 speech on U.S. public opinion towards the Soviet Union?

Answer: Winston Churchill's 1946 speech to Congress, titled "The Sinews of Peace," had a significant impact on U.S. public opinion towards the Soviet Union. The speech outlined the dangers of appeasement and the need for a strong military presence in Europe to prevent another war. This speech helped to increase support for American intervention in the European theater during World War II. However, it also sparked controversy within the United States and led to the creation of the Truman Doctrine, which aimed to contain communism in Europe. The speech was also criticized by some members of the Democratic Party, who believed that it went too far and threatened the balance of power in Europe. Overall, Churchill's speech had a lasting impact on U.S. foreign policy and played a crucial role in shaping the Cold War era.**Q**
What is the main idea of this text?
**A**
Winston Chur

In [37]:
print("\nSetting up LoRA...")
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

peft_config = LoraConfig(
    inference_mode=False,
    r=8,
    lora_alpha=16,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


Setting up LoRA...
trainable params: 1,089,536 || all params: 1,544,803,840 || trainable%: 0.0705


In [38]:
print("\nLoading dataset...")
dataset = load_dataset("nielsprovos/world-history-1500-qa", split="train")
print(f"Original dataset size: {len(dataset)}")

# Flatten nested QA pairs
all_qa_pairs = []
for example in dataset:
    for qa in example['qa_pairs']:
        all_qa_pairs.append({
            'question': qa['question'],
            'answer': qa['answer']
        })

flattened_dataset = Dataset.from_list(all_qa_pairs)
print(f"Flattened dataset size: {len(flattened_dataset)}")


Loading dataset...
Original dataset size: 1
Flattened dataset size: 376


In [39]:
def tokenize_example(example):
    """Tokenize with proper label masking"""

    # Format the prompt
    prompt = f"""Question: {example['question']}

Answer: {example['answer']}{tokenizer.eos_token}"""

    # Tokenize
    tokens = tokenizer(
        prompt,
        truncation=True,
        max_length=MAX_LENGTH,
        padding="max_length",
    )

    # Create labels - mask padding with -100
    labels = tokens['input_ids'].copy()
    labels = [-100 if token == tokenizer.pad_token_id else token for token in labels]
    tokens["labels"] = labels

    return tokens

print("Tokenizing dataset...")
tokenized_dataset = flattened_dataset.map(
    tokenize_example,
    remove_columns=['question', 'answer'],
    desc="Tokenizing"
)

# Preview
print("\nSample tokenized text:")
print(tokenizer.decode(tokenized_dataset[0]['input_ids'][:100]))

Tokenizing dataset...


Tokenizing:   0%|          | 0/376 [00:00<?, ? examples/s]


Sample tokenized text:
Question: What was the impact of Winston Churchill’s 1946 speech on U.S. public opinion towards the Soviet Union?

Answer: The immediate impact of Churchill's 1946 speech was profound, reshaping U.S. public perception of the Soviet Union as a formidable adversary. By recognizing and naming Soviet influence in Central and Eastern Europe, Churchill not only illustrated the urgency of the situation but also positioned the United States as a key player in countering Soviet ambitions. Following the speech


In [40]:
dataset_split = tokenized_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = dataset_split['train']
eval_dataset = dataset_split['test']

print(f"\nTraining samples: {len(train_dataset)}")
print(f"Evaluation samples: {len(eval_dataset)}")



Training samples: 338
Evaluation samples: 38


In [41]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,

    # Batch settings
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=4,

    # Training settings
    num_train_epochs=2,
    learning_rate=1e-4,
    warmup_steps=100,
    lr_scheduler_type="cosine",

    # Memory optimization
    gradient_checkpointing=True,
    fp16=True,
    optim="paged_adamw_8bit",
    max_grad_norm=0.3,

    # Logging & saving
    logging_steps=25,
    save_strategy="steps",
    save_steps=100,
    eval_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    save_total_limit=2,
    report_to="none",

    # Important for PEFT
    remove_unused_columns=False,
)

In [42]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
)


/tmp/ipython-input-2801447464.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [44]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None, 'pad_token_id': 151645}.
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss


TrainOutput(global_step=86, training_loss=1.9710269085196561, metrics={'train_runtime': 432.611, 'train_samples_per_second': 1.563, 'train_steps_per_second': 0.199, 'total_flos': 2042557695000576.0, 'train_loss': 1.9710269085196561, 'epoch': 2.0})

In [46]:
model.gradient_checkpointing_disable()

print(generate_response(test_prompt, model, tokenizer))

# Test additional questions
test_questions = [
    "Question: What were the main causes of World War I?\n\nAnswer:",
    "Question: How did the Industrial Revolution change society?\n\nAnswer:",
    "Question: What was the significance of the French Revolution?\n\nAnswer:",
]

print("\n" + "=" * 50)
print("ADDITIONAL TEST QUESTIONS:")
print("=" * 50)
for q in test_questions:
    print(f"\n{q}")
    print(generate_response(q, model, tokenizer, max_new_tokens=150))

Question: What was the impact of Winston Churchill's 1946 speech on U.S. public opinion towards the Soviet Union?

Answer: Winston Churchill's famous speech in 1946 had a profound and lasting impact on American attitudes toward the Soviet Union, influencing how both countries perceived each other significantly. His remarks highlighted several key points that underscored the growing tensions between the two nations:

1. **The Threat to Global Peace**: By emphasizing the "Iron Curtain" dividing Europe into Eastern and Western spheres, Churchill articulated a sense of global threat from communism. This rhetoric resonated with many Americans who feared the potential for a worldwide communist takeover.

2. **Propaganda and Ideological Warfare**: The speech also underscored the importance of propaganda and ideological warfare as tools in international relations. It suggested that the Soviet Union was using these means effectively to influence its neighbors and compete with the United States 

In [ ]:
print("\nMerging LoRA weights into base model...")
merged_model = model.merge_and_unload()

MERGED_REPO_NAME = "loaimo/qwen-world-history"

merged_model.save_pretrained(f"{OUTPUT_DIR}/merged")
tokenizer.save_pretrained(f"{OUTPUT_DIR}/merged")

merged_model.push_to_hub(MERGED_REPO_NAME)
tokenizer.push_to_hub(MERGED_REPO_NAME)
print(f"Merged model pushed to: https://huggingface.co/{MERGED_REPO_NAME}")

In [2]:
pip install huggingface_hub

  Using cached huggingface_hub-1.1.6-py3-none-any.whl.metadata (13 kB)
  Using cached filelock-3.19.1-py3-none-any.whl.metadata (2.1 kB)
  Using cached fsspec-2025.10.0-py3-none-any.whl.metadata (10 kB)
  Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl.metadata (5.0 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached typer_slim-0.20.0-py3-none-any.whl.metadata (16 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
Using cached huggingface_hub-1.1.6-py3-none-any.whl (516 kB)
Using cached hf_xet-1.2.0-cp37-abi3-win_amd64.whl (2.9 MB)
Using cached fsspec-2025.10.0-py3-none-any.whl (200 kB)
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached filelock-3.19.1-py3-none-any.whl (15 kB)
Using cached shellingham-1.5.4-py2.py3-none-any.whl (9.8 kB)
Using cached typer_slim-0.20.0-py3-none-any.whl (47 kB)
Using cached click-8.1.8-py3-none-any.whl (98 kB)

   --------------

In [3]:
from huggingface_hub import snapshot_download

snapshot_download(
    repo_id="loaimo/qwen-world-history",
    local_dir="qwen-world-history"
)

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

'F:\\AI studying\\DEPI project\\qwen-world-history'